In [20]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

def extracao(filename):
    Vet = []
    image = cv2.imread(filename)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray,(5,5),0)
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 135, 20)
    for i in range(15):
        for k in range(10):
            crop_img = (thresh[25+i*200:(i+1)*200-25, 15+k*200:(k+1)*200-15])
            crop_img = cv2.resize(crop_img,dsize=(28,28),interpolation=cv2.INTER_AREA)
            
            # padrao de entrada para o classificador
            crop_img = np.reshape(crop_img, (784,))
            Vet.append(crop_img)
    return(Vet)

Vet = []
Target = []

for i in range(6):
    for k in range(15):
        for j in range(10):
            Target.append(j)
    Vet = Vet + extracao('Folha (%i).jpg'%(i+1))

np.save('Num.npy',Vet)
np.save('Target.npy',Target)